In [26]:
!pip install tensorflow

In [27]:
import tensorflow as tf
import numpy as np
import random
from tensorflow.contrib import rnn
import os
import zipfile
from scipy import spatial
import collections
import math
import time
import pickle
from pathlib import Path
from six.moves.urllib.request import urlretrieve
from tensorflow.contrib.tensorboard.plugins import projector

In [28]:
data_path = 'C://Users//willi10l//Downloads//'

In [29]:
#hyperparameters

vocab_fraction = .00005
hidden_layer = 512
embedding_dim = 100
time_steps = 3
learning_rate = .001

#training parameters

num_iters = 10000
display_steps = 500

log_dir = 'C://Users//willi10l/Downloads'
run_name = 'rundemo -' + str(vocab_fraction) + '-' + str(hidden_layer) + '-' + str(num_iters)


In [30]:
zip_path = 'C://Users//willi10l//Downloads//glove.6B.zip'
pkl_path = 'C://Users//willi10l//Downloads//glove.6B.pkl'

def create_corpus(zip_path, pkl_path):
    vocab = []
    with zipfile.ZipFile(zip_path) as f1:
        with f1.open(f1.namelist()[0]) as bfile:
            all_data = bfile.read().strip().split()
        
            for word in all_data:
                vocab.append(word.decode('UTF-8'))
                
#serialize the data and put it into a pickle file which can be retrieved later

    with open(pkl_path, 'wb') as f2:
        pickle.dump(vocab, f2, pickle.HIGHEST_PROTOCOL)
        
    return vocab

if Path(pkl_path).is_file():
    
    with open(pkl_path, 'rb') as f:
        corpus = pickle.load(f)
        print("Pickle file loaded")
        
else:
    corpus = create_corpus(zip_path, pkl_path)
    
trunc_corpus = corpus[:int(math.floor(len(corpus) * vocab_fraction))]

if 'the' in trunc_corpus:
    print("'the' found")
    
print(len(trunc_corpus))
            

Pickle file loaded
'the' found
1020


In [31]:
print(trunc_corpus)

['the', '0.418', '0.24968', '-0.41242', '0.1217', '0.34527', '-0.044457', '-0.49688', '-0.17862', '-0.00066023', '-0.6566', '0.27843', '-0.14767', '-0.55677', '0.14658', '-0.0095095', '0.011658', '0.10204', '-0.12792', '-0.8443', '-0.12181', '-0.016801', '-0.33279', '-0.1552', '-0.23131', '-0.19181', '-1.8823', '-0.76746', '0.099051', '-0.42125', '-0.19526', '4.0071', '-0.18594', '-0.52287', '-0.31681', '0.00059213', '0.0074449', '0.17778', '-0.15897', '0.012041', '-0.054223', '-0.29871', '-0.15749', '-0.34758', '-0.045637', '-0.44251', '0.18785', '0.0027849', '-0.18411', '-0.11514', '-0.78581', ',', '0.013441', '0.23682', '-0.16899', '0.40951', '0.63812', '0.47709', '-0.42852', '-0.55641', '-0.364', '-0.23938', '0.13001', '-0.063734', '-0.39575', '-0.48162', '0.23291', '0.090201', '-0.13324', '0.078639', '-0.41634', '-0.15428', '0.10068', '0.48891', '0.31226', '-0.1252', '-0.037512', '-1.5179', '0.12612', '-0.02442', '-0.042961', '-0.28351', '3.5416', '-0.11956', '-0.014533', '-0.1499

In [32]:
glove_dir = 'C://Users//willi10l//Downloads//'
glove_file = 'glove.6B.100d'
glove_path = glove_dir + glove_file + '.txt'

with open (glove_path) as myfile:
    head = [next(myfile) for x in range(3)]
print(head[0])

the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459 0.28787 -0.06731 0.30906 -0.26384 -0.13231 -0.20757 0.33395 -0.33848 -0.31743 -0.48336 0.1464 -0.37304 0.34577 0.052041 0.44946 -0.46971 0.02628 -0.54155 -0.15518 -0.14107 -0.039722 0.28277 0.14393 0.23464 -0.31021 0.086173 0.20397 0.52624 0.17164 -0.082378 -0.71787 -0.41531 0.20335 -0.12763 0.41367 0.55187 0.57908 -0.33477 -0.36559 -0.54857 -0.062892 0.26584 0.30205 0.99775 -0.80481 -3.0243 0.01254 -0.36942 2.2167 0.72201 -0.24978 0.92136 0.034514 0.46745 1.1079 -0.19358 -0.074575 0.23353 -0.052062 -0.22044 0.057162 -0.15806 -0.30798 -0.41625 0.37972 0.15006 -0.53212 -0.2055 -1.2526 0.071624 0.70565 0.49744 -0.42063 0.26148 -1.538 -0.30223 -0.073438 -0.28312 0.37104 -0.25217 0.016215 -0.017099 -0.38984 0.87424 -0.72569 -0.51058 -0.52028 -0.1459 0.8278 0.27062



In [33]:
#create vocab from teh GloVe embedding file

vocab_pkl = glove_dir + glove_file + '.vpkl'
embed_pkl = glove_dir + glove_file + '.mpkl'

def create_glove_vocab(path, v_pkl, e_pkl):
    vocab = []
    embedding_dict = {}
    
    with open(path, 'r', encoding="utf-8") as f:
        for line in f.readlines():
            data = line.strip().split(' ')
            vocab.append(data[0])
            embedding_dict[data[0]] = [float(val) for val in data [1:]]
            
    with open(v_pkl, 'wb') as f1:
        pickle.dump(vocab, f1, pickle.HIGHEST_PROTOCOL)
        print("created vocab file")
        
    with open(e_pkl, 'wb') as f2:
        pickle.dump(embedding_dict, f2, pickle.HIGHEST_PROTOCOL)
        print("created embed file")
        
    return vocab, embedding_dict

if Path(vocab_pkl).is_file() and Path(embed_pkl).is_file():
    print("found files")
    with open(vocab_pkl, 'rb') as f1:
        glove_vocab = pickle.load(f1)
        print("loaded vocab file")
        
    with open(embed_pkl, 'rb') as f2:
        embedding_dict = pickle.load(f2)
        print("loaded embed file")
        
else:
    glove_vocab, embedding_dict = create_glove_vocab(glove_path, vocab_pkl, embed_pkl)
    
    
print(glove_vocab[:1])
print(len(glove_vocab))         

found files
loaded vocab file
loaded embed file
['the']
400000


In [34]:
#create dictionary of training corpus

def create_dictionary(training_data):
    dictionary = {}
    count = collections.Counter(training_data). most_common()
    for word, _ in count:
        dictionary[word] = len(dictionary.items())
    reverse_dict = dict(zip(dictionary.values(), dictionary.keys()))
    
    return dictionary, reverse_dict

dictionary, reverse_dict = create_dictionary(data_path)
print(len(dictionary))

17


In [35]:
#some words in corpus may not have embeddings in GloVe. So we create random initial embeddings for those words
#create embeddings specific to training vocab

def create_embeds(training_dict, glove_vocab, embedding_dict):
    input_embeds = []
    exceptions = {}
    for word, i in training_dict.items():
        if word in glove_vocab:
            input_embeds.append(embedding_dict[word])
        else:
            new_embed = np.random.uniform(low=-0.2, high=0.2, size=embedding_dim)
            input_embeds.append(new_embed)
            exceptions.append(word)
        return input_embeds, exceptions
    
input_embeds, exceptions = create_embeds(dictionary, glove_vocab, embedding_dict)
input_embeds = np.asarray(input_embeds)

#creating a spatial representation of the embeddings. This can be queried
tree = spatial.KDTree(input_embeds)

print(len(exceptions))



0


In [37]:
#Create LSTM model in tensorflow
tf.reset_default_graph()

with tf.name_scope('inputs'):
    
    x_id = tf.placeholder(dtype=tf.int32, shape=(None, time_steps), name='input_words')
    y_label = tf.placeholder(dtype=tf.float32, shape=(None, embedding_dim), name='target_embed')
    
with tf.name_scope('outputs'):
    out_wts = tf.Variable(tf.random_normal(shape=([hidden_layer, embedding_dim])), name='out_wts')
    out_biases = tf.Variable(tf.random_normal(shape=({embedding_dim})), name='out_biases')

with tf.name_scope('embedding'):
    
    #make the embedding trainable
    
    embed_wts = tf.Variable(tf.constant(0.0, shape=[len(dictionary), embedding_dim]), trainable=True, name='embed_wts')
    embeddings = tf.placeholder(dtype=tf.float32, shape=(len(dictionary), embedding_dim), name='input_embeds')
    
    #initialize with glove + home made embeddings
    
    embed_init = embed_wts.assign(embeddings)
    
    input_embed = tf.nn.embedding_lookup(embed_wts, x_id)
    
with tf.name_scope('LSTM_model'):
    
    #this is a 2 layer LSTM
    
    lstm_cell = rnn.BasicLSTMCell(hidden_layer)
    lstm_cell = rnnBasicLSTMCell(hidden_layer)
    
    rnn_cell = rnn.MultiRNNCell({lstm_cell1, lstm_cell2})
    
    #the inputs are basically time step number of vector (in this case 3
    #we untack them in to a matrix of time-step, embed_dim
    
    x_id_unstack = tf.unstack(input_embed, time_steps, 1)
    
    outputs, state = tf.nn.static_rnn(rnn_cell, x_id_unstack, dtype=tf.float32)
    
with tf.name_scope('loss'):
    
    #define the loss
    
    out_pred = tf.matmul(outputs[-1], out_wts) + out_biases
    loss = tf.reduce_mean(tf.nn.l2_loss(out_pred - y_label))
    
tf.summary.scalar("12_loss", loss)
with tf.name_scope('train_fn'):
    
    train_step = tf.train.AdamsOptimizer(learning_rate=learning_rate).minimize(loss)
    

TypeError: Failed to convert object of type <class 'set'> to Tensor. Contents: {100}. Consider casting elements to a supported type.

In [ ]:
#train the model

summ = tf.summary.merge_all()

start = time.time()

training_data = trunc_corpus

init = tf.global_variables_initializer()
sess = tf.Session()
train_writer = tf.summery.FileWriter(log_dir + run_name, sess.graph)
sess.run(init)
sess.run(embed_init, fedd_dict={embeddings: input_embeds})

steps = 0
step_loss = 0
acc_loss = 0
offset = random.randint(0, time_steps + 1)
end_offset = offset + time_steps

while steps < num_iter:
    if offset > len(training_data) - end_offset:
        offset = random.randit(0, time_steps +1)
        
    x_int = [dictionary[training_data[i]] for i in range(offset, offset + timee_steps)] 
    
    #index of the input word
    x_int = np.reshape(x_int, [-1, time_steps])
    
    y_pos = offset + time_steps #index the prediction word
    y_int = dictionary[training_data[y_post]] #index of the word in the dictionary
    y_embed = input_embeds[y_int, :] #getting the embedding vector of y which is a list
    y_embed = np.reshape(y_embed, [1, -1]) #the list has to be converted into a numpy array for tf to work 
    _, step_loss, pred, summary = sess.run([train_step, loss, out_pred, summ] feed_dict={x_id: x_int, y_label: y_embed})
    train_writer.add_summary(summary, steps)
    
    acc_loss += step_loss
    
    if (steps + 1) % display_steps ==0:
        input_words = [training_data[i] for i in range(offset, offset + time_steps)]
        target_word = {training_data{y_pos}}
        
    #using KDTree to query for the ids of words which are closest in vector space of the predicted embeddings
    
        nearest_dist, nearest_idx = tree.query(pred{0}, 3)
        nearest_words = {reverse_dict{i} fori in nearest_idx}
        
        if (steps + 1) == display_steps:
            print(pred)
            
        print("Input words [%s} vs target %s vs nearest words [%s)" % (input_words, target_words, nearest_words))
        
        acc_loss = 0
        
    steps += 1
    offset += time_steps + 1
    
#setup the embeddings projector on tensorboard
config = projector.ProjectorConfig()
embedding_config = config.embeddings.add()
embedding_config.tensor_name = 'embeddings/embed_wts' #the name space is very important
embedding_conf.metadata_path = os.path.join('/home/ubuntu/nbs/data/demo/', 'metadata.tsv')
projector.visualize_embeddings(train_writer, config)

#save the model
saver = tf.train.Saver()
saver.save(sess, os.path.join('/home/ubuntu/nbs/data/demo', "model.ckpt"))

end = time.time()
print(end - start)